## **Plot**

In [12]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt

In [66]:
with open("results/INSAR_20200909_PSDS_v3/PATCH_1/pscands.1.ll", "r") as f:
    ll = np.loadtxt(f)


In [ ]:
# Plot the lon/lat points
plt.figure(figsize=(20,10))
plt.scatter(ll[:,0], ll[:,1], s=1, alpha=0.5)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('PS Candidate Locations')
plt.grid(True)
plt.show()

# Split into lon/lat arrays for later use
lon = ll[:,0] 
lat = ll[:,1]


In [70]:
pm1 = sio.loadmat("results/INSAR_20200909_PSDS_v3/PATCH_1/pm1.mat")
ps2 = sio.loadmat("results/INSAR_20200909_PSDS_v3/ph2.mat")
ij = ps2['ps']['ij'][0][0].astype(int)
lonlat = ps2['ps']['lonlat'][0][0]
selected_ph = pm1['ph_patch'][ij[:, 0], :]


In [21]:
ph2 = sio.loadmat("results/INSAR_20200909_PSDS_v3/ph2.mat")
ps2 = ps2 = sio.loadmat("results/INSAR_20200909_PSDS_v3/ps2.mat")
ij = ps2['ij'].astype(int) - 1
lonlat = ps2['lonlat']
selected_ph = ph2['ph'][ij[:, 0], :]

In [ ]:
# Plot selected phase values at their pixel locations
plt.figure(figsize=(20,12))

# Use viridis colormap with increased point size and alpha
plt.scatter(lonlat[:, 0], lonlat[:, 1], 
           c=np.angle(selected_ph[:,0]),
           cmap='viridis', 
           s=1,  # Increased marker size
           alpha=0.7)  # Increased opacity

# Add a more visible colorbar
cbar = plt.colorbar(label='Phase (rad)')
cbar.ax.tick_params(labelsize=12)

# Enhance axis labels and title
plt.xlabel('Range pixel', fontsize=14)
plt.ylabel('Azimuth pixel', fontsize=14)
plt.title('Selected Phase Values at Pixel Locations', fontsize=16, pad=15)

# Add grid for better readability
plt.grid(True, alpha=0.3)

plt.show()



In [ ]:
# Get number of interferograms
n_ifg = pm1['ph_patch'].shape[1]

# Create subplot grid
n_rows = int(np.ceil(np.sqrt(n_ifg)))
n_cols = int(np.ceil(n_ifg / n_rows))

plt.figure(figsize=(4*n_cols, 4*n_rows))

# Plot phase for each interferogram
for i in range(n_ifg):
    plt.subplot(n_rows, n_cols, i+1)
    plt.imshow(np.angle(pm1['ph_patch'][:,i]).reshape(-1,1), aspect='auto', cmap='hsv')
    plt.colorbar(label='Phase (rad)')
    plt.title(f'Interferogram {i+1}')
    plt.xlabel('PS Point')

plt.tight_layout()
plt.show()


## **Data**

In [1]:
import numpy as np
from modules.tomo.input_parm import Input

In [ ]:
filepath = "results/INSAR_20200909_PSDS_v3/diff0/20200909_20200723.psds"
with open(filepath, "rb") as f:
    data = np.fromfile(f, dtype=np.complex64)
data.reshape((401, 2117))
data


In [16]:
filepath = "results/INSAR_20200909_PSDS_v3/shp.npz"
shp_data = np.load(filepath, allow_pickle=True)['shp'].item()['patches']

In [18]:
for idx, item in enumerate(shp_data):
    print(f'{idx}: {item["BroNum"].shape}')

0: (21, 75)
1: (21, 75)
2: (21, 75)
3: (21, 75)
4: (21, 75)
5: (21, 75)
6: (21, 75)
7: (21, 75)
8: (21, 75)
9: (21, 75)
10: (21, 75)
11: (21, 75)
12: (21, 75)
13: (21, 75)
14: (21, 75)
15: (21, 75)
16: (21, 75)
17: (21, 75)
18: (21, 75)
19: (21, 75)
20: (21, 75)
21: (21, 75)
22: (21, 75)
23: (21, 75)
24: (21, 75)
25: (21, 75)
26: (21, 17)
27: (21, 75)
28: (21, 75)
29: (21, 75)
30: (21, 75)
31: (21, 75)
32: (21, 75)
33: (21, 75)
34: (21, 75)
35: (21, 75)
36: (21, 75)
37: (21, 75)
38: (21, 75)
39: (21, 75)
40: (21, 75)
41: (21, 75)
42: (21, 75)
43: (21, 75)
44: (21, 75)
45: (21, 75)
46: (21, 75)
47: (21, 75)
48: (21, 75)
49: (21, 75)
50: (21, 17)
51: (21, 75)
52: (21, 75)
53: (21, 75)
54: (21, 75)
55: (21, 75)
56: (21, 75)
57: (21, 75)
58: (21, 75)
59: (21, 75)
60: (21, 75)
61: (21, 75)
62: (21, 75)
63: (21, 75)
64: (21, 75)
65: (21, 75)
66: (21, 75)
67: (21, 75)
68: (21, 75)
69: (21, 75)
70: (21, 75)
71: (21, 75)
72: (21, 75)
73: (21, 75)
74: (21, 75)
75: (21, 75)
76: (21, 75)
77: (21, 

In [8]:
slc, interf = Input([7, 25], output=False).run()

-> PSDS Enabled
-> Unified ComSAR Disabled


In [9]:
slc['datastack'][0]['datastack'].shape

(21, 75, 5)

In [19]:
filepath = "results/INSAR_20200909_PSDS_v3/coherence.npz"
coherence_data = np.load(filepath, allow_pickle=True)['coherence']

for idx, item in enumerate(coherence_data):
    print(f'{idx}: {item["coherence"].shape}')








0: (5, 5, 21, 75)
1: (5, 5, 21, 75)
2: (5, 5, 21, 75)
3: (5, 5, 21, 75)
4: (5, 5, 21, 75)
5: (5, 5, 21, 75)
6: (5, 5, 21, 75)
7: (5, 5, 21, 75)
8: (5, 5, 21, 75)
9: (5, 5, 21, 75)
10: (5, 5, 21, 75)
11: (5, 5, 21, 75)
12: (5, 5, 21, 75)
13: (5, 5, 21, 75)
14: (5, 5, 21, 75)
15: (5, 5, 21, 75)
16: (5, 5, 21, 75)
17: (5, 5, 21, 75)
18: (5, 5, 21, 75)
19: (5, 5, 21, 75)
20: (5, 5, 21, 75)
21: (5, 5, 21, 75)
22: (5, 5, 21, 75)
23: (5, 5, 21, 75)
24: (5, 5, 21, 75)
25: (5, 5, 21, 75)
26: (5, 5, 21, 75)
27: (5, 5, 21, 75)
28: (5, 5, 21, 17)
29: (5, 5, 21, 75)
30: (5, 5, 21, 75)
31: (5, 5, 21, 75)
32: (5, 5, 21, 75)
33: (5, 5, 21, 75)
34: (5, 5, 21, 75)
35: (5, 5, 21, 75)
36: (5, 5, 21, 75)
37: (5, 5, 21, 75)
38: (5, 5, 21, 75)
39: (5, 5, 21, 75)
40: (5, 5, 21, 75)
41: (5, 5, 21, 75)
42: (5, 5, 21, 75)
43: (5, 5, 21, 75)
44: (5, 5, 21, 75)
45: (5, 5, 21, 75)
46: (5, 5, 21, 75)
47: (5, 5, 21, 75)
48: (5, 5, 21, 75)
49: (5, 5, 21, 75)
50: (5, 5, 21, 75)
51: (5, 5, 21, 75)
52: (5, 5, 21, 75)
53: